Note: DeepMIMO data: BS16, row3500_3516, 3.4 GHz

In [21]:
import torch
import torch.nn as nn


import os
import sys
import numpy as np
from scipy.io import savemat

# Add the Torch_code directory to the Python path
# sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath('../helper'))
import config
import utils
import loader
import plotfig
# import skfuzzy as fuzz

In [22]:
# FILE_PATH = os.path.dirname(os.path.abspath("__file__"))
# print(FILE_PATH)
# print(config.temp_path)
# print(config.FILE_PATH)

In [23]:
# Configuration
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 32 #64  # Batch size
NUM_EPOCHS = 20 # 20

# rows from DeepMIMO dataset settings
# change rows according to the .mat dataset file 
rows = [['3500', '3516']] 
fc = '3p4' #Hz can change to '60'
rowss = "3500_3516"
learning_rate = 0.00001 # 1e-5
SNR = np.arange(0, 31, 5) # 0:5:30 dB
outer_file_path = os.path.abspath(os.path.join(config.FILE_PATH, 
                                                '..', 'DeepMIMOv2', 'DeepMIMO_Data', 'Static_BS16', 'freq_symb_1ant_612sub_ver4'))


In [24]:
# create readme.txt file
content = """Generated by file 'train/static_CNN_lr1e-5_v4_r3500_3516_3p4_instance_min_max.ipynb'.
Correspond with BS16, 3.4 GHz fc, rows 3500_3516,
DeepMIMOv2/DeepMIMO_Dta_Static_BS16/freq_sym_1ant_612sub_ver4,
Using min-max scaler for each sample"""

norm_approach = 'minmax' # can be set to 'std'

# Paths to save
idx_save_path = loader.find_incremental_filename('../model/static/CNN/BS16/'+ rowss,'ver', '_', '')
model_path = '../model/static/CNN/BS16/' + rowss + '/ver' + str(idx_save_path) + '_/readme.txt'
figure_path = '../figure/static/CNN/BS16/' + rowss + '/ver' + str(idx_save_path) + '_/readme.txt'

if not os.path.exists(os.path.dirname(model_path)):
    os.makedirs(os.path.dirname(model_path))
if not os.path.exists(os.path.dirname(figure_path)):
    os.makedirs(os.path.dirname(figure_path))

# Open the file in write mode ('w'). If the file does not exist, it will be created.
with open(model_path, 'w') as file:
    # Write the content to the file
    file.write(content)

with open(figure_path, 'w') as file:
    # Write the content to the file
    file.write(content)

print(f"File '{model_path}' and ' {figure_path} ' created and content written.")

save_folder_model = os.path.join(config.FILE_PATH, 'model/static/CNN', 'BS16', rowss, 'ver' + str(idx_save_path) + '_')
save_folder_fig = os.path.join(config.FILE_PATH, 'figure', 'static', 'CNN', 'BS16' ,  rowss, 'ver' + str(idx_save_path) +'_')

File '../model/static/CNN/BS16/3500_3516/ver4_/readme.txt' and ' ../figure/static/CNN/BS16/3500_3516/ver4_/readme.txt ' created and content written.


In [25]:
SNR

array([ 0,  5, 10, 15, 20, 25, 30])

In [26]:
# snr = 0
# [trainLabels, valLabels], [H_equal_train, H_linear_train, H_practical_train], [H_equal_val, H_linear_val, H_practical_val] = loader.load_data(outer_file_path, rows, fc, device, snr)


In [27]:
for snr in SNR:
    print(f" SNR: {snr}/{SNR[-1]}")

    [trainLabels, valLabels], [H_equal_train, H_linear_train, H_practical_train], [H_equal_val, H_linear_val, H_practical_val] = loader.load_data(outer_file_path, rows, fc, device, snr)

    # -----------------------------------------------------
    # 1. When input is H_linear (after LS+LI)
    print(f" Training for LS+LI")
    # [samples, 2, 612, 14]
    train_loader, trainLabel_min, trainLabel_max = loader.genLoader(H_linear_train, trainLabels, BATCH_SIZE, device, 'train', True, norm_approach)
    val_loader,     valLabel_min,   valLabel_max = loader.genLoader(H_linear_val,     valLabels, BATCH_SIZE, device, 'valid', False, norm_approach)
        # no shuffle in validation so valLabel_min and valLable_max remain the min and max arrays
        # of valLabels
        
    # model
    model = utils.CNN_Est().to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
    criterion = nn.MSELoss()

    # 1.5 Training loop
    train_loss =[]
    val_loss = []
    H_NN_val = torch.empty_like(valLabels) # [nVal, 2, 612, 14]
    min_H_true = []
    max_H_true = []
    num_epochs = NUM_EPOCHS
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        if (epoch == num_epochs-1):
            i = 0
        for inputs, targets, targets_min, targets_max in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        avg_train_loss = running_loss / len(train_loader)
        train_loss.append(avg_train_loss)
        print(f"SNR: {snr}/{SNR[-1]}, LS+LI, Epoch {epoch+1}/{num_epochs}, Loss: {avg_train_loss} ")
        
        # Validation 
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for val_inputs, val_targets, val_targetsMin, val_targetsMax in val_loader:
                val_inputs_real = val_inputs[:,0,:,:].unsqueeze(1)
                val_inputs_imag = val_inputs[:,1,:,:].unsqueeze(1)
                val_targets_real = val_targets[:,0,:,:].unsqueeze(1)
                val_targets_imag = val_targets[:,1,:,:].unsqueeze(1)
                
                val_outputs_real = model(val_inputs_real)
                val_loss_real = criterion(val_outputs_real, val_targets_real)
                running_val_loss += val_loss_real.item()
                
                val_outputs_imag = model(val_inputs_imag)
                val_loss_imag = criterion(val_outputs_imag, val_targets_imag)
                running_val_loss += val_loss_imag.item()
                
                if (epoch == num_epochs-1): # the results after the last training 
                    H_NN_val[i:i+val_outputs_real.size(0),0,:,:].unsqueeze(1).copy_(val_outputs_real)
                    H_NN_val[i:i+val_outputs_imag.size(0),1,:,:].unsqueeze(1).copy_(val_outputs_imag)
                    
                    i = i+val_outputs_imag.size(0)       
                    
                
        avg_val_loss = running_val_loss / (len(val_loader)*2)
        val_loss.append(avg_val_loss)    
                
        print(f"SNR: {snr}/{SNR[-1]}, LS+LI, Val Loss: {avg_val_loss}")


    save_folder = os.path.join(save_folder_model, str(snr)+'dB')
    os.makedirs(save_folder, exist_ok=True)
    index_save = loader.find_incremental_filename(save_folder, 'CNN_', '_variable')

    model_save_path = os.path.join(save_folder,  'CNN_' +str(index_save)+'_LS_LI_CNN_model.pth')
    variable_save_path = os.path.join(save_folder, 'CNN_' +str(index_save)+'_variable.pth')
    params_save_path = os.path.join(save_folder, 'CNN_' +str(index_save)+'_params.mat')
    
    params = {   
                'SNR': snr,
                'epoc': NUM_EPOCHS,
                'rows': rowss,
                'learning_rate': learning_rate,
                'train_track_LI': train_loss,
                'val_track_LI': val_loss,
    }
    variables = {             
                'train_track_LI': train_loss,
                'val_track_LI': val_loss,
                # 'train_min_LI': trainData_min.cpu(),
                # 'train_max_LI': trainData_max.cpu(),
                # 'train_label_min': trainLabels_min.cpu(),
                # 'train_label_max': trainLabels_max.cpu(),
    }
    # variable to save 
    # Save the models' state dictionaries 
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
        }, model_save_path)

    figure_save_path = os.path.join(save_folder_fig, str(snr) + 'dB') 
    
    os.makedirs(figure_save_path, exist_ok=True)

    plotfig.figLoss(train_loss, val_loss, index_save, figure_save_path, '_LS_LI_Loss.png')


    # True channel
    H_val_true = valLabels.cpu()
    # convert to complex matrices
    H_val_true_complex = torch.complex(H_val_true[:,0,:,:], H_val_true[:,1,:,:])
    # variables['H_val_true'] = H_val_true # (nVal, 2, 612, 14)

    plotfig.figTrueChan(H_val_true[-1,0,:,:], 'True Channel', index_save, figure_save_path, '_trueChannel.png')

    # Linear interpolated channel
    H_val_linInterp = H_linear_val.cpu()
    # convert to complex matrices
    H_val_linInterp_complex = torch.complex(H_val_linInterp[:,0,:,:], H_val_linInterp[:,1,:,:])

    # NMSE of Linear Interpolation
    # Calculate the mean squared error
    mse_LI = torch.mean(torch.abs(H_val_true_complex - H_val_linInterp_complex) ** 2)
    # Calculate the variance of the reference tensor (complex_tensor1)
    calculate variance of each target sample 
    variance = torch.var(H_val_true_complex)
    # Calculate the NMSE
    nmse_LI = mse_LI / variance
    variables['NMSE_LI'] = nmse_LI.cpu()
    print(f"LS+LI NMSE: {nmse_LI.item()}")
    
    plotfig.figPredChan(H_val_linInterp[-1,0,:,:], 'LS + Interpolate Estimated Channel',
                            nmse_LI, index_save, figure_save_path, '_LS_LI_estimatedChan.png')

    # Estimated Channel 
    H_val_NN = H_NN_val.cpu()    
    plotfig.figTrueChan(H_val_NN[-1,0,:,:], 'LI+CNN Estimated Channel (before de-normlized)', 
                            index_save, figure_save_path, '_LS_LI_CNN_estimatedChan_before_denorm.png')

    # De-normalized                                                               
    H_val_NN_denormd = utils.deMinMax(H_NN_val, valLabel_min, valLabel_max)
                        #     H_NN_val == [nVal, 2, 612, 14] 
                        # valLabel_min == [nVal,1]
                        
    H_val_NN_denormd = H_val_NN_denormd.cpu()
    # variables['H_val_LI_NN'] = H_val_NN_denormd # (nVal, 2, 612, 14)

    # convert to complex matrices
    H_val_NN_denormd_complex = torch.complex(H_val_NN_denormd[:,0,:,:], H_val_NN_denormd[:,1,:,:])

    # NMSE of Linear Interpolation + NN
    # Calculate the mean squared error
    mse_LI_NN = torch.mean(torch.abs(H_val_true_complex - H_val_NN_denormd_complex) ** 2)
    # Calculate the NMSE
    nmse_LI_NN = mse_LI_NN / variance
    print(f"LS+LI+CNN NMSE: {nmse_LI_NN.item()}")
    variables['NMSE_LI_NN'] = nmse_LI_NN.cpu()
    
    plotfig.figPredChan(H_val_NN_denormd[-1,0,:,:], 'LI+CNN Estimated Channel (after de-normlized)',
                            nmse_LI_NN, index_save, figure_save_path, '_LS_LI_CNN_estimatedChan.png')


##########################################
    # ------------------------------------------------------
    # When Input of the NN is just H_equalized
    print(f" Training for LS")
    # [samples, 2, 612, 14]
    H_LS_train = H_equal_train.cpu()
    plotfig.figTrueChan(H_LS_train[0,0,:,:], 'LS Channel', index_save, figure_save_path, '_LS_Chan.png')
    
    # Split into training and validation sets for H_NN training
    train_loader, trainLabel_min, trainLabel_max = loader.genLoader(H_equal_train, trainLabels, BATCH_SIZE, device, 'train',  True, norm_approach)
    val_loader,     valLabel_min,   vallabel_max = loader.genLoader(H_equal_val,     valLabels, BATCH_SIZE, device, 'valid', False, norm_approach)


    model2 = utils.CNN_Est().to(device)
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=learning_rate) 
    criterion = nn.MSELoss()

    # Training loop
    train_loss =[]
    val_loss = []
    H_NN_val = torch.empty_like(valLabels) # [nVal, 2, 612, 14]
    num_epochs = NUM_EPOCHS
    for epoch in range(num_epochs):
        model2.train()
        running_loss = 0.0
        if (epoch == num_epochs-1):
            i = 0
        for inputs, targets, targets_min, targets_max in train_loader:
            optimizer2.zero_grad()
            outputs = model2(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer2.step()
            running_loss += loss.item()
            
        avg_train_loss = running_loss / len(train_loader)
        train_loss.append(avg_train_loss)
        print(f"SNR: {snr}/{SNR[-1]}, LS, Epoch {epoch+1}/{num_epochs}, Loss: {avg_train_loss} ")
        
        # Validation 
        model2.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for val_inputs, val_targets, val_targetsMin, val_targetsMax in val_loader:
                val_inputs_real = val_inputs[:,0,:,:].unsqueeze(1)
                val_inputs_imag = val_inputs[:,1,:,:].unsqueeze(1)
                val_targets_real = val_targets[:,0,:,:].unsqueeze(1)
                val_targets_imag = val_targets[:,1,:,:].unsqueeze(1)
                
                val_outputs_real = model2(val_inputs_real)
                val_loss_real = criterion(val_outputs_real, val_targets_real)
                running_val_loss += val_loss_real.item()
                
                val_outputs_imag = model2(val_inputs_imag)
                val_loss_imag = criterion(val_outputs_imag, val_targets_imag)
                running_val_loss += val_loss_imag.item()
                
                if (epoch == num_epochs-1):
                    H_NN_val[i:i+val_outputs_real.size(0),0,:,:].unsqueeze(1).copy_(val_outputs_real)
                    H_NN_val[i:i+val_outputs_imag.size(0),1,:,:].unsqueeze(1).copy_(val_outputs_imag)
                    i = i+val_outputs_imag.size(0)
                
        avg_val_loss = running_val_loss / (len(val_loader)*2)
        val_loss.append(avg_val_loss)    
                
        print(f"SNR: {snr}/{SNR[-1]}, LS, Val Loss: {avg_val_loss}")

    plotfig.figLoss(train_loss, val_loss, index_save, figure_save_path, '_LS_Loss.png')

    # De-normalized                                                                
    H_val_NN_denormd = utils.deMinMax(H_NN_val, valLabel_min, valLabel_max)
                        #     H_NN_val == [nVal, 2, 612, 14] 
                        # valLabel_min == [nVal,1]
    H_val_NN_denormd = H_val_NN_denormd.cpu()

    model_save_path = os.path.join(save_folder,  'CNN_' +str(index_save)+'_LS_CNN_model.pth')

    # variables['H_val_LS_NN']= H_val_NN_denormd.cpu() # (nVal, 2, 612, 14)
    variables['train_track_LS']= train_loss
    variables['val_track_LS']= val_loss

    # Save parameters
    params['train_track_LS']= train_loss
    params['val_track_LS']= val_loss
    savemat(params_save_path, params)
    # Save the models' state dictionaries 
    torch.save({'model_state_dict': model2.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
                }, model_save_path)


    # NMSE of LS + NN
    H_val_LS_NN_complex = torch.complex(H_val_NN_denormd[:,0,:,:], H_val_NN_denormd[:,1,:,:])
    # Calculate the mean squared error
    mse_LS_NN = torch.mean(torch.abs(H_val_true_complex - H_val_LS_NN_complex) ** 2)
    # Calculate the NMSE
    nmse_LS_NN = mse_LS_NN / variance
    print(f"LS+CNN NMSE: {nmse_LS_NN.item()}")
    variables['NMSE_LS_NN'] = nmse_LS_NN.cpu()
    
    plotfig.figPredChan(H_val_NN_denormd[-1,0,:,:], 'LS+CNN Estimated Channel (after de-normlized)',
                            nmse_LS_NN, index_save, figure_save_path, '_LS_CNN_estimatedChan.png')
    

    torch.save( variables,variable_save_path)

 SNR: 0/30
 Training for LS+LI
SNR: 0/30, LS+LI, Epoch 1/20, Loss: 0.18383328772561494 
SNR: 0/30, LS+LI, Val Loss: 0.050281547009944916
SNR: 0/30, LS+LI, Epoch 2/20, Loss: 0.04864660806434099 
SNR: 0/30, LS+LI, Val Loss: 0.03799135669727217
SNR: 0/30, LS+LI, Epoch 3/20, Loss: 0.034907598622403176 
SNR: 0/30, LS+LI, Val Loss: 0.02491906052455306
SNR: 0/30, LS+LI, Epoch 4/20, Loss: 0.02815565404070671 
SNR: 0/30, LS+LI, Val Loss: 0.02327575898644599
SNR: 0/30, LS+LI, Epoch 5/20, Loss: 0.027440981425041724 
SNR: 0/30, LS+LI, Val Loss: 0.02263125927526165
SNR: 0/30, LS+LI, Epoch 6/20, Loss: 0.027188914465219823 
SNR: 0/30, LS+LI, Val Loss: 0.022362281737679787
SNR: 0/30, LS+LI, Epoch 7/20, Loss: 0.027138715658640098 
SNR: 0/30, LS+LI, Val Loss: 0.02233770371160724
SNR: 0/30, LS+LI, Epoch 8/20, Loss: 0.026825778457054566 
SNR: 0/30, LS+LI, Val Loss: 0.02249901470812884
SNR: 0/30, LS+LI, Epoch 9/20, Loss: 0.026767239618916496 
SNR: 0/30, LS+LI, Val Loss: 0.023643837437372316
SNR: 0/30, LS+L

/home/thien/Hprediction/one_shot_Hest_cleanver/Torch_code/helper/plotfig.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10, 5))


SNR: 10/30, LS, Epoch 1/20, Loss: 0.29672583452490875 
SNR: 10/30, LS, Val Loss: 0.2337618659843098
SNR: 10/30, LS, Epoch 2/20, Loss: 0.2007665978094866 
SNR: 10/30, LS, Val Loss: 0.14792067591439595
SNR: 10/30, LS, Epoch 3/20, Loss: 0.0563722331683303 
SNR: 10/30, LS, Val Loss: 0.01901371099732139
SNR: 10/30, LS, Epoch 4/20, Loss: 0.016189324707442592 
SNR: 10/30, LS, Val Loss: 0.013680250722576271
SNR: 10/30, LS, Epoch 5/20, Loss: 0.012226142000060442 
SNR: 10/30, LS, Val Loss: 0.010562592016702349
SNR: 10/30, LS, Epoch 6/20, Loss: 0.009445728054595029 
SNR: 10/30, LS, Val Loss: 0.008094178385693918
SNR: 10/30, LS, Epoch 7/20, Loss: 0.007219504093924581 
SNR: 10/30, LS, Val Loss: 0.006117593729868531
SNR: 10/30, LS, Epoch 8/20, Loss: 0.005472173297041377 
SNR: 10/30, LS, Val Loss: 0.0046720166309652
SNR: 10/30, LS, Epoch 9/20, Loss: 0.004227956158628817 
SNR: 10/30, LS, Val Loss: 0.0036183127773587
SNR: 10/30, LS, Epoch 10/20, Loss: 0.0034121970610888025 
SNR: 10/30, LS, Val Loss: 0.

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

## Add code for generating NMSE over SNR